In [ ]:
# ===== 0. INSTALL =========================================
!pip install -q pillow opencv-python moviepy tqdm supervision \
               torchaudio torchvision noisereduce \
               git+https://github.com/roboflow/rf-detr.git \
               accelerate openai

print("Зависимости готовы (используем системный PyTorch).")

---

In [ ]:
from google.colab import drive
from google.colab import files, widgets
import torch, cv2, moviepy.editor as mpy
from tqdm.auto import tqdm
from rfdetr import RFDETRBase
from pathlib import Path
import os, re, json, requests, base64, numpy as np, soundfile as sf, librosa
import textwrap
from collections import defaultdict


In [2]:
drive.mount('/content/drive')

WEIGHTS_PATH = '/content/drive/MyDrive/Colab Notebooks/rf_detr_803.pt'   # вес модели
RESULT_DIR   = '/content/drive/MyDrive/Colab Notebooks/foul_results'     # куда класть вывод
!mkdir -p "$RESULT_DIR"

print("Пути настроены")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Пути настроены


----

In [3]:
device_str = 'cuda' if torch.cuda.is_available() else 'cpu'

model = RFDETRBase(pretrain_weights=WEIGHTS_PATH, device=device_str)
model.optimize_for_inference()
print("✅  Модель загрузилась на", device_str)

reinitializing detection head with 2 classes


Loading pretrain weights


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


















✅  Модель загрузилась на cuda


----

In [4]:
from google.colab import files, widgets

print("Выберите видео (.mp4)")
up = files.upload()
assert len(up) == 1, "Нужно загрузить 1 файл"
vid_name = next(iter(up))
VID_PATH = f"/content/{vid_name}"
print("✅ Видео на диске:", VID_PATH)

# VideoCapture / Writer
cap  = cv2.VideoCapture(VID_PATH)
fps  = cap.get(cv2.CAP_PROP_FPS) or 25
W    = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
H    = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
OUT_PATH = VID_PATH.replace(".mp4", "_annotated.mp4")
writer   = cv2.VideoWriter(OUT_PATH, fourcc, fps, (W, H))


Выберите видео (.mp4)


Saving foull.mp4 to foull.mp4
✅ Видео на диске: /content/foull.mp4


----

In [5]:
RESULT_DIR = "/content/drive/MyDrive/Colab Notebooks/foul_results"
os.makedirs(RESULT_DIR, exist_ok=True)

OUT_PATH   = f"{RESULT_DIR}/{Path(VID_PATH).stem}_annotated.mp4"

# папка для кадров
FRAME_DIR  = f"{RESULT_DIR}/frames"
os.makedirs(FRAME_DIR, exist_ok=True)

print("Результаты будут писаться в:", RESULT_DIR)


✅  Результаты будут писаться в: /content/drive/MyDrive/Colab Notebooks/foul_results


----

In [6]:
import os, json, requests, base64, numpy as np, soundfile as sf, librosa

DEEP_KEY   = "9kScTQSpFJCd6OdxblR8bHjw3iWSPZkV"
WSPR_URL   = "https://api.deepinfra.com/v1/inference/openai/whisper-large"

AUDIO_PATH = VID_PATH.replace(".mp4", ".wav")
!ffmpeg -loglevel error -y -i "$VID_PATH" -vn -ac 1 -ar 16000 "$AUDIO_PATH"

audio_b64 = base64.b64encode(open(AUDIO_PATH, "rb").read()).decode()
resp = requests.post(WSPR_URL,
        headers={"Authorization": f"Bearer {DEEP_KEY}"},
        json={"audio": audio_b64, "language": "en", "return_segments": True},
        timeout=300).json()

with open(VID_PATH.replace(".mp4", "_transcript.json"), "w") as f:
    json.dump(resp, f, indent=2, ensure_ascii=False)

print("✅ Whisper ok; пример:", resp["segments"][0]["text"][:60])

# -----
y, sr = sf.read(AUDIO_PATH, dtype="float32")
S     = np.abs(librosa.stft(y, n_fft=512, hop_length=256))**2
freqs = librosa.fft_frequencies(sr=sr, n_fft=512)
band  = (freqs > 2500) & (freqs < 4500)
energy= S[band].mean(axis=0)
thr   = energy.mean() + 3*energy.std()
wh_secs = {int(i*256/sr) for i,e in enumerate(energy) if e > thr}

print(f"✅ Простых свистков найдено: {len(wh_secs)} секунд.")



✅ Whisper ok; пример:  It's two of three tonight.
✅ Простых свистков найдено: 5 секунд.


----

In [7]:
DEEPINFRA_API_KEY = "9kScTQSpFJCd6OdxblR8bHjw3iWSPZkV"
CHAT_URL          = "https://api.deepinfra.com/v1/openai/chat/completions"
MODEL_ID          = "meta-llama/Meta-Llama-3-70B-Instruct"
HEADERS_LLM = {"Authorization": f"Bearer {DEEPINFRA_API_KEY}",
               "Content-Type": "application/json"}

if "trn" not in globals():
    with open(VID_PATH.replace(".mp4", "_transcript.json")) as f:
        trn = json.load(f)
segments = trn.get("segments", [])
wh_secs  = globals().get("wh_secs", set())

if not segments:
    print("⚠️  Стенограмма пуста → calls = []")
    calls = []
else:
    WIN, STEP = 8.0, 4.0
    windows, t = [], 0.0
    total = segments[-1]["end"]
    while t < total:
        t_end = t + WIN
        txt = " ".join(s["text"].strip()
                       for s in segments
                       if s["start"] < t_end and s["end"] > t)
        windows.append((t, t_end, txt))
        t += STEP


    SYS_PROMPT = ("You are an expert basketball referee analyst. "
                  "Respond with ONE word only:\n"
                  "• foul\n• no_foul\n• other")

    def ask_llm(text_block: str) -> str:
        payload = {
            "model": MODEL_ID,
            "messages": [
                {"role": "system", "content": SYS_PROMPT},
                {"role": "user",   "content": text_block[:900]}
            ],
            "temperature": 0,
            "max_tokens": 3
        }
        r = requests.post(CHAT_URL, json=payload, headers=HEADERS_LLM, timeout=60)
        r.raise_for_status()
        word = re.split(r"\W+", r.json()["choices"][0]["message"]["content"].lower())[0]
        return word if word in ("foul", "no_foul") else "other"


    KW = re.compile(r"\b(foul|фол|play on|no foul)\b", re.I)
    calls = []
    for st, ed, txt in windows:
        mid = int((st + ed) // 2)
        if KW.search(txt) or mid in wh_secs:
            lbl = ask_llm(txt)
            if lbl != "other":
                calls.append({"start": st, "end": ed, "label": lbl, "text": txt})

    if not calls:
        print("ℹ️  fallback: scanning first 90 s")
        for st, ed, txt in windows:
            if st > 90: break
            lbl = ask_llm(txt)
            if lbl != "other":
                calls.append({"start": st, "end": ed, "label": lbl, "text": txt})

print(f"✅ calls: {len(calls)}")



✅ calls: 1


----

In [8]:
import cv2, numpy as np, os
from collections import defaultdict

SCORE_TH = 0.4
saved_secs = set()

cap  = cv2.VideoCapture(VID_PATH)
fps  = cap.get(cv2.CAP_PROP_FPS) or 25
W, H = int(cap.get(3)), int(cap.get(4))

writer = cv2.VideoWriter(
    OUT_PATH,
    cv2.VideoWriter_fourcc(*"mp4v"),
    fps, (W, H)
)

foul_secs = set()
frame_idx = 0

while True:
    ok, frame = cap.read()
    if not ok:
        break

    rgb  = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    dets = model.predict(rgb, conf_thres=SCORE_TH)

    sec_mark = int(frame_idx / fps)
    foul_here = False

    for box, conf, cls in zip(dets.xyxy, dets.confidence, dets.class_id):
        if cls != 1:
            continue
        foul_here = True
        x1, y1, x2, y2 = map(int, box)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
        cv2.putText(frame, f"foul {conf:.2f}", (x1, y1 - 6),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1)

    if foul_here:
        foul_secs.add(sec_mark)
        if sec_mark not in saved_secs:
            jpg_path = f"{FRAME_DIR}/foul_{sec_mark:06d}.jpg"
            cv2.imwrite(jpg_path, frame)
            saved_secs.add(sec_mark)

    writer.write(frame)
    frame_idx += 1

cap.release(); writer.release()
print(f"✅ Сохранены кадры: {len(saved_secs)} шт. → {FRAME_DIR}")

episodes = []
if foul_secs:
    cur_s = prev_s = min(foul_secs)
    for s in sorted(foul_secs):
        if s == prev_s or s == prev_s + 1:
            prev_s = s
        else:
            episodes.append((cur_s, prev_s))
            cur_s = prev_s = s
    episodes.append((cur_s, prev_s))

VOICE_END = globals().get("VOICE_END",
            (trn["segments"][-1]["end"] if segments else 0))

episodes_filt = [(s, e) for s, e in episodes if s <= VOICE_END + 3]
print("✅ эпизодов после кластеризации:", len(episodes_filt))


✅ Сохранены кадры: 7 шт. → /content/drive/MyDrive/Colab Notebooks/foul_results/frames
✅ эпизодов после кластеризации: 1


----

In [9]:
import pandas as pd, numpy as np, textwrap

def iou_time(a,b):
    inter = max(0, min(a[1], b[1]) - max(a[0], b[0]))
    union = max(a[1], b[1]) - min(a[0], b[0])
    return inter / union if union else 0

rows, used_calls, matches = [], set(), set()


for i, (s, e) in enumerate(episodes_filt):

    match_call = None
    for j, c in enumerate(calls):
        if j in used_calls: continue
        if iou_time((s, e), (c["start"], c["end"])) > .3:
            match_call = (j, c); used_calls.add(j); break

    whistle = any(t in wh_secs for t in range(s, e+1))
    match   = bool(match_call) or whistle
    ref_lbl = (match_call[1]["label"] if match_call else
               ("whistle" if whistle else "—"))
    phrase  = match_call[1]["text"] if match_call else ""

    rows.append(dict(ep_start=s, ep_end=e,
                     model="foul",
                     referee=ref_lbl,
                     phrase=phrase[:120],
                     match=match))
    if match: matches.add(i)


for j,c in enumerate(calls):
    if j not in used_calls and c["label"]=="foul":
        rows.append(dict(ep_start=int(c["start"]),
                         ep_end=int(c["end"]),
                         model="—",
                         referee="foul",
                         phrase=c["text"][:120],
                         match=False))

df = pd.DataFrame(rows).sort_values("ep_start").reset_index(drop=True)


pd.set_option('display.max_rows', None)
print(df.to_string(index=False))
pd.reset_option('display.max_rows')


tp = sum((r.match and r.model=="foul") for r in df.itertuples())
fp = sum((r.model=="foul" and not r.match) for r in df.itertuples())
fn = sum((r.referee in ("foul","whistle") and not r.match) for r in df.itertuples())

precision = tp / (tp + fp + 1e-9)
recall    = tp / (tp + fn + 1e-9)
f1        = 2*precision*recall/(precision+recall+1e-9)

print(f"\nPrecision {precision:.2%}   Recall {recall:.2%}   F1 {f1:.2%}")

print(textwrap.dedent(f"""
    ──────────────────────────────────────────────────────────────
    **Что показывает таблица**

    • ep_start / ep_end  – диапазон эпизода (секунды видео).
    • model              – решение модели (здесь всегда 'foul').
    • referee            – что слышно в аудио:
        'foul'     – комментатор/рефери явно объявил фол;
        'no_foul'  – наоборот, «без фола» / «play on»;
        'whistle'  – слышен свисток, но слов нет;
        '—'        – аудио ничего не сообщает.
    • phrase             – сама фраза, если есть (обрезана 120 симв).
    • match (True/False) – подтверждён ли эпизод аудио (фразой **или** свистком).

    **Метрики**

    Precision = TP / (TP + FP) — доля модельных эпизодов, подтверждённых аудио.
    Recall    = TP / (TP + FN) — доля аудио-фолов, которые модель тоже нашла.
    F1        — гармоническое среднее precision и recall.

    Где TP — эпизоды, совпавшие c фразой 'foul' (IoU > 0.3) **или** содержащие свисток.
    FP — “лишние” эпизоды модели, FN — фразы 'foul', без перекрывающего эпизода.
    ──────────────────────────────────────────────────────────────
"""))


 ep_start  ep_end model referee                                                                                                                   phrase  match
        1       7  foul    foul It's two of three tonight. They're not going away. Brooklyn will battle. Dennis Smith Jr. with a knockdown of Butler, an   True

Precision 100.00%   Recall 100.00%   F1 100.00%

──────────────────────────────────────────────────────────────
**Что показывает таблица**

• ep_start / ep_end  – диапазон эпизода (секунды видео).  
• model              – решение модели (здесь всегда 'foul').  
• referee            – что слышно в аудио:
    'foul'     – комментатор/рефери явно объявил фол;
    'no_foul'  – наоборот, «без фола» / «play on»;
    'whistle'  – слышен свисток, но слов нет;
    '—'        – аудио ничего не сообщает.
• phrase             – сама фраза, если есть (обрезана 120 симв).  
• match (True/False) – подтверждён ли эпизод аудио (фразой **или** свистком).

**Метрики**

Precision = TP / (TP

----